In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import scipy
import scipy.io as sio
import scipy.io.wavfile

In [ ]:
# Load code challenge file
challenge = scipy.io.loadmat('filtering_codeChallenge.mat')

challenge # contains sampling rate (fs), original data (x), filtered data (y)

In [ ]:
# Extract variables from MAT file

fs = challenge['fs'][0][0] # 1000 Hz

 # remember to squeeze so that the shape is 1d array (x,) vs. 2d array (x,1). The fft results in wrong values if the input array is not 1d
x = np.squeeze(challenge['x'])
y = np.squeeze(challenge['y']) 

print(x.shape)
print(y.shape)

npnts = len(x)

In [ ]:
# Plot in Time domain

time_vec = np.arange(0,npnts)/fs 
hz_vec = np.linspace(0,fs/2,int(np.floor(npnts/2)+1))
x_pwr = np.abs(scipy.fft.fft(x)/npnts)**2
y_pwr = np.abs(scipy.fft.fft(y)/npnts)**2

# Time Domain
plt.subplot(211)
plt.plot(time_vec, x, label='original ->  x')
plt.plot(time_vec, y, label='filtered -> y')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (a.u.)')
plt.title('Time Domain')
plt.legend()

# Frequency Domain
plt.subplot(212)
plt.plot(hz_vec, x_pwr[:len(hz_vec)], label='x power')
plt.plot(hz_vec, y_pwr[:len(hz_vec)], label='y power')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.title('Frequency Domain')
plt.xlim([0,40])
plt.legend()
plt.subplots_adjust(hspace=1)




In [ ]:
hp_cutoff = 5
order = int(np.round(11*fs/hp_cutoff)+1)
print(order)

frex = [0, 5, 5.5, fs/2] # frequency bands of interest (transition width)
bshape = [0, 0, 1, 1] # Shape for high pass filter

# Make high pass filter 
hp_filter = signal.firls(order, bands=frex,desired=bshape, fs=fs)

# Evaluate filter characteristics
plt.subplot(121)
plt.plot(hp_filter)

plt.subplot(122)
hz_hpfilt = np.linspace(0,fs,order)
plt.plot(frex, bshape, label='ideal')
filtpow = np.abs(scipy.fft.fft(hp_filter))**2 # Not dividing by order to get max to be 1
plt.plot(hz_hpfilt, filtpow, label='actual')  
plt.xlim([2, 7])
plt.legend()

In [ ]:
x_hp = signal.filtfilt(hp_filter, 1, x) # apply filter

plt.plot(time_vec, x, label='original')
plt.plot(time_vec, x_hp, label='high-pass filtered')
plt.legend()
plt.title('Time Domain')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (a.u.)')

x_hp_pwr = np.abs(scipy.fft.fft(x_hp)/npnts)**2
plt.figure()
plt.plot(hz_vec, x_pwr[:len(hz_vec)], label='original')
plt.plot(hz_vec, x_hp_pwr[:len(hz_vec)], label='high-pass filtered')
plt.xlim([0,40])
plt.title('Frequency Domain')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.legend()


In [ ]:
# Now low-pass filter the data

lp_cutoff = 30 # Hz
lp_transw = 5
lp_bands = [0, lp_cutoff, lp_cutoff+lp_transw, fs/2]
lp_shape = [1, 1, 0, 0]
lp_order = int(np.round(10*fs/lp_cutoff)+2)
hz_lpfilt = np.linspace(0,fs,lp_order)
print(lp_order)

lp_filter = signal.firls(lp_order,bands=lp_bands, desired=lp_shape, fs=fs)

# Evaluate filter 
plt.subplot(121)
plt.plot(lp_filter)

plt.subplot(122)
plt.plot(lp_bands, lp_shape, label='ideal')
plt.plot(hz_lpfilt, np.abs(scipy.fft.fft(lp_filter))**2, label='actual')
plt.xlim([27,40])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain')
plt.legend()

In [ ]:
x_hp_lp = signal.filtfilt(lp_filter,1,x_hp) # apply low-pass filter on previously high-pass filtered data
x_hp_lp_pwr = np.abs(scipy.fft.fft(x_hp_lp)/npnts)**2


plt.plot(time_vec, x, label='original')
plt.plot(time_vec, x_hp_lp, label='2-stage filtered')
plt.title('Time Domain')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()


plt.figure()
#plt.plot(hz_vec, x_pwr[:len(hz_vec)], label='original')
plt.plot(hz_vec, x_hp_lp_pwr[:len(hz_vec)], label='2-stage filtered')
plt.plot(hz_vec, y_pwr[:len(hz_vec)], label='Goal')
plt.title('Frequency Domain')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.legend()
plt.xlim([0, 40])

In [ ]:
# Now notch filter the data

frange = [17, 25]
notch_order = int(np.round(21*fs/frange[0]))
notch_hz_vec = np.linspace(0,fs,notch_order)
print(notch_order)

notch_filter = signal.firwin(notch_order, cutoff=frange, pass_zero = 'bandstop', fs=fs, window='hanning')
# Evaluate notch filter
plt.subplot(121)
plt.plot(notch_filter)

plt.subplot(122)
plt.plot(notch_hz_vec, np.abs(scipy.fft.fft(notch_filter))**2, label='actual')
plt.plot([0, frange[0], frange[0], frange[1], frange[1], fs/2], [1, 1, 0, 0, 1, 1], label='ideal')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain')
plt.xlim([10,30])

In [ ]:
# Now apply the filter
x_final = signal.filtfilt(notch_filter, 1, x_hp_lp)

# Get Power spectra
x_final_pwr = np.abs(scipy.fft.fft(x_final)/npnts)**2

plt.plot(time_vec, x_final, label='Filtered')
plt.plot(time_vec, y, label='Goal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Time Domain')
plt.legend()

plt.figure()
plt.plot(hz_vec, y_pwr[:len(hz_vec)], label='Goal')
plt.plot(hz_vec, x_final_pwr[:len(hz_vec)], label='Filtered')
plt.title('Frequency Domain')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.legend()
plt.xlim([0, 40])

In [ ]:
np.corrcoef(y_pwr, x_final_pwr)